In [64]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer

# Composição do dataset

In [22]:
df_meta_dados = pd.read_csv('../dados/meta_dados_estacoes.csv')
df_meta_dados.head()

,ano,mes,data,hora,id_estacao,radiacao_global,temperatura_bulbo_hora,temperatura_orvalho_hora,temperatura_max,temperatura_min,temperatura_orvalho_max,temperatura_orvalho_min,umidade_rel_max,umidade_rel_min,umidade_rel_hora
0,2023,4,2023-04-09,14:00:00,A373,1148.9,24.1,24.0,24.9,24.1,24.4,23.7,99.0,95.0,99.0
1,2023,4,2023-04-09,05:00:00,A419,NaN,22.3,22.2,22.4,22.3,22.2,22.1,99.0,99.0,99.0
2,2023,12,2023-12-21,23:00:00,A329,NaN,23.6,23.4,24.1,23.6,23.4,22.7,99.0,93.0,99.0
3,2023,12,2023-12-21,15:00:00,A450,972.1,23.6,23.3,24.8,23.5,24.3,23.0,99.0,94.0,99.0
4,2023,12,2023-12-21,15:00:00,A369,2249.1,31.5,31.4,31.7,28.4,31.7,30.0,100.0,99.0,99.0


Vamos utilizar apenas a temperatura máxima e mínima do dia

In [23]:
# remover colunas desnecessárias
df_meta_dados_novo = df_meta_dados.drop(columns=['radiacao_global', 'temperatura_bulbo_hora', 'temperatura_orvalho_hora',
                                            'temperatura_orvalho_max', 'temperatura_orvalho_min', 'umidade_rel_max',
                                            'umidade_rel_min', 'umidade_rel_hora','mes','ano'])

df_meta_dados_novo.head()

,data,hora,id_estacao,temperatura_max,temperatura_min
0,2023-04-09,14:00:00,A373,24.9,24.1
1,2023-04-09,05:00:00,A419,22.4,22.3
2,2023-12-21,23:00:00,A329,24.1,23.6
3,2023-12-21,15:00:00,A450,24.8,23.5
4,2023-12-21,15:00:00,A369,31.7,28.4


In [24]:
# Agrupar por dia pegando a temperatura máxima e mínima
df_meta_dados_novo = df_meta_dados_novo.groupby(['id_estacao','data']).agg({'temperatura_max':'max','temperatura_min':'min'}).reset_index()
df_meta_dados_novo.head()

,id_estacao,data,temperatura_max,temperatura_min
0,A305,2023-01-01,32.4,25.9
1,A305,2023-01-02,32.6,24.4
2,A305,2023-01-03,32.6,24.6
3,A305,2023-01-04,32.7,25.2
4,A305,2023-01-05,32.9,25.5


In [25]:
# Varificando se as estações possuem todos os dias de 2023
df_meta_dados_novo['id_estacao'].value_counts().loc[lambda x : x < 364]

Series([], Name: id_estacao, dtype: int64)

Todas as estações possuem uma contagem de pelo menos 364 dias

In [26]:
df_meta_dados_novo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30576 entries, 0 to 30575
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id_estacao       30576 non-null  object 
 1   data             30576 non-null  object 
 2   temperatura_max  26142 non-null  float64
 3   temperatura_min  26142 non-null  float64
dtypes: float64(2), object(2)
memory usage: 955.6+ KB


Temos uma quantidade muito grande de valores nulos, vamos criar um modelo para tentar inferir esses valores

In [45]:
# Utilizando o dataset completo
import basedosdados as bd

query = """
SELECT
    ano,
    mes,
    data,
    hora,
    id_estacao,
    precipitacao_total,
    pressao_atm_hora,
    pressao_atm_max,
    pressao_atm_min,
    radiacao_global,
    temperatura_bulbo_hora,
    temperatura_orvalho_hora,
    temperatura_max,
    temperatura_min,
    temperatura_orvalho_max,
    temperatura_orvalho_min,
    umidade_rel_max,
    umidade_rel_min,
    umidade_rel_hora,
    vento_direcao,
    vento_rajada_max,	
    vento_velocidade

FROM
    `basedosdados.br_inmet_bdmep.microdados`
WHERE
    ano = 2023
    AND id_estacao IN (
        'A323', 'A327', 'A371', 'A408', 'A412', 'A413', 'A415', 'A416', 'A418',
        'A423', 'A424', 'A425', 'A426', 'A428', 'A429', 'A430', 'A432', 'A433',
        'A435', 'A436', 'A439', 'A440', 'A441', 'A442', 'A443', 'A448', 'A449',
        'A450', 'A458', 'A305', 'A306', 'A314', 'A315', 'A319', 'A324', 'A325',
        'A332', 'A339', 'A342', 'A347', 'A358', 'A359', 'A360', 'A368', 'A369',
        'A310', 'A313', 'A321', 'A333', 'A334', 'A348', 'A373', 'A307', 'A309',
        'A322', 'A328', 'A329', 'A349', 'A350', 'A351', 'A366', 'A370', 'A308',
        'A330', 'A331', 'A336', 'A337', 'A343', 'A345', 'A354', 'A365', 'A316',
        'A317', 'A318', 'A340', 'A367', 'A372', 'A417', 'A419', 'A420', 'A451',
        'A453', 'A526', 'A539', 'A543', 'A559', 'A563', 'A454'
    );
"""

df_meta_dados = bd.read_sql(query, billing_project_id="projetopdi-430718")
# Salvar o dataset
df_meta_dados.to_csv('../dados/meta_dados_completo.csv', index=False)
df_meta_dados.head()

Downloading: 100%|██████████| 733824/733824 [03:11<00:00, 3829.47rows/s]


,ano,mes,data,hora,id_estacao,precipitacao_total,pressao_atm_hora,pressao_atm_max,pressao_atm_min,radiacao_global,...,temperatura_max,temperatura_min,temperatura_orvalho_max,temperatura_orvalho_min,umidade_rel_max,umidade_rel_min,umidade_rel_hora,vento_direcao,vento_rajada_max,vento_velocidade
0,2023,4,2023-04-09,09:00:00,A367,0.0,987.4,987.4,986.5,2.2,...,23.8,23.3,23.6,23.1,99.0,99.0,99.0,66.0,1.5,0.8
1,2023,4,2023-04-09,08:00:00,A367,0.0,986.5,986.5,986.1,NaN,...,23.4,23.2,23.2,23.0,99.0,99.0,99.0,282.0,0.9,0.4
2,2023,4,2023-04-09,06:00:00,A367,0.0,985.7,985.8,985.6,NaN,...,23.8,23.6,23.5,23.3,99.0,98.0,99.0,289.0,1.7,0.4
3,2023,4,2023-04-09,04:00:00,A313,0.0,950.8,951.4,950.8,NaN,...,23.1,22.9,22.8,22.6,99.0,98.0,99.0,173.0,3.8,0.4
4,2023,4,2023-04-09,07:00:00,A313,0.0,951.4,951.5,950.8,NaN,...,22.9,22.8,22.7,22.6,99.0,99.0,99.0,163.0,3.8,1.4


In [67]:
df_meta_dados = pd.read_csv('../dados/meta_dados_completo.csv')
df_meta_dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 733824 entries, 0 to 733823
Data columns (total 22 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ano                       733824 non-null  int64  
 1   mes                       733824 non-null  int64  
 2   data                      733824 non-null  object 
 3   hora                      733824 non-null  object 
 4   id_estacao                733824 non-null  object 
 5   precipitacao_total        510665 non-null  float64
 6   pressao_atm_hora          588148 non-null  float64
 7   pressao_atm_max           587454 non-null  float64
 8   pressao_atm_min           587451 non-null  float64
 9   radiacao_global           339095 non-null  float64
 10  temperatura_bulbo_hora    585996 non-null  float64
 11  temperatura_orvalho_hora  551192 non-null  float64
 12  temperatura_max           585332 non-null  float64
 13  temperatura_min           585328 non-null  f

In [68]:
# Converter data pro formato datetime
df_meta_dados['data'] = pd.to_datetime(df_meta_dados['data'], format='%Y-%m-%d')

# Criar a coluna com apenas o dia
df_meta_dados['dia'] = df_meta_dados['data'].dt.day

# Converter hora pra inteiro
df_meta_dados['hora'] = df_meta_dados['hora'].str.replace(':','').astype(int)
df_meta_dados['hora'] = df_meta_dados['hora']/10000
# Codificação cíclica da hora
df_meta_dados['hora_sin'] = np.sin(2 * np.pi * df_meta_dados['hora'] / 24)
df_meta_dados['hora_cos'] = np.cos(2 * np.pi * df_meta_dados['hora'] / 24)

# Inicializar o codificador de rótulos
label_encoder = LabelEncoder()

# Aplicar a codificação de rótulos
df_meta_dados['id_estacao_encoded'] = label_encoder.fit_transform(df_meta_dados['id_estacao'])

# Removendo colunas desnecessárias
df_meta_dados = df_meta_dados.drop(columns=['data','ano','id_estacao'])
df_meta_dados = df_meta_dados.dropna()
df_meta_dados.head()

,mes,hora,precipitacao_total,pressao_atm_hora,pressao_atm_max,pressao_atm_min,radiacao_global,temperatura_bulbo_hora,temperatura_orvalho_hora,temperatura_max,...,umidade_rel_max,umidade_rel_min,umidade_rel_hora,vento_direcao,vento_rajada_max,vento_velocidade,dia,hora_sin,hora_cos,id_estacao_encoded
0,4,9.0,0.0,987.4,987.4,986.5,2.2,23.7,23.6,23.8,...,99.0,99.0,99.0,66.0,1.5,0.8,9,0.707107,-0.707107,44
5,4,9.0,0.0,952.7,952.7,951.8,74.5,22.8,22.7,23.0,...,99.0,99.0,99.0,73.0,3.2,1.6,9,0.707107,-0.707107,6
6,12,9.0,0.2,986.9,986.9,986.3,36.6,23.2,23.1,23.2,...,99.0,99.0,99.0,164.0,1.6,0.2,21,0.707107,-0.707107,22
13,4,9.0,0.0,986.6,986.6,986.4,47.3,21.8,21.7,21.8,...,99.0,99.0,99.0,357.0,1.3,0.3,11,0.707107,-0.707107,44
16,12,16.0,0.0,968.4,969.5,968.3,3728.5,35.9,10.4,36.1,...,25.0,21.0,21.0,75.0,14.6,3.4,23,-0.866025,-0.500000,32


In [69]:
# Separar variáveis independentes (X) e dependente (y)
X = df_meta_dados.drop(columns=['temperatura_max'])
y = df_meta_dados['temperatura_max']

# Dividir em conjuntos de treino e teste, mantendo os NaNs apenas no treino
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Regressão Linear

In [71]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [72]:
# Escalonar os dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Treinar o modelo de regressão linear
model = LinearRegression()
model.fit(X_train_scaled, y_train)

# Prever e avaliar o modelo no conjunto de teste
y_pred = model.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mse)

print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {mae}')
print(f'R² Score: {r2}')
print(f'Root Mean Squared Error: {rmse}')

Mean Squared Error: 0.1584054241687905
Mean Absolute Error: 0.28693011876005486
R² Score: 0.9926498876227411
Root Mean Squared Error: 0.39800178915275053


**Mean Squared Error (MSE): 0.158**: MSE mede a média dos quadrados dos erros. Um valor menor indica melhor desempenho. Nesse caso, 0.158 é um valor relativamente baixo.

**Mean Absolute Error (MAE): 0.287**: MAE mede a média dos erros absolutos. Um MAE de 0.287 é bastante baixo, indicando que as previsões do modelo estão, em média, a 0.287 unidades de distância dos valores reais.

**R² Score: 0.993**: O R², ou coeficiente de determinação, mede a proporção da variância na variável dependente que é previsível a partir das variáveis independentes. Um R² de 0.993 significa que 99.3% da variância na variável dependente é explicada pelas variáveis independentes no modelo. Isso indica um ajuste excelente.

**Root Mean Squared Error (RMSE): 0.398**: RMSE é a raiz quadrada do MSE e fornece uma medida do erro na mesma unidade que a variável dependente. Um RMSE de 0.398 também é relativamente baixo.

# Random Forest

In [75]:
rf = RandomForestRegressor(random_state=42)

# treinar o modelo
rf.fit(X_train_scaled, y_train)

# prever no conjunto de teste
y_pred_rf = rf.predict(X_test_scaled)

# avaliar o modelo
mse_rf = mean_squared_error(y_test, y_pred_rf)
mae_rf = mean_absolute_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)
rmse_rf = np.sqrt(mse_rf)

print(f'Random Forest Mean Squared Error: {mse_rf}')
print(f'Random Forest Mean Absolute Error: {mae_rf}')
print(f'Random Forest R² Score: {r2_rf}')
print(f'Random Forest Root Mean Squared Error: {rmse_rf}')

Random Forest Mean Squared Error: 0.23679467845394403
Random Forest Mean Absolute Error: 0.32864038933252404
Random Forest R² Score: 0.9890125763930987
Random Forest Root Mean Squared Error: 0.4866155345382472
